In [1]:
import pandas as pd
import numpy as np
import re

In [217]:
polugodie = pd.read_csv("Полугодия.csv", index_col="year")
u_sin = pd.read_csv("У-син.csv", index_col="Орган")
season_qi = pd.read_csv("season_qi.csv", index_col="Орган")
qi = pd.read_csv("qi.csv", index_col="Стихия")
stvoly = pd.read_csv("stvoly.csv", index_col="year")
vetvi = pd.read_csv("vetvi.csv", index_col="year")
sloy = pd.read_csv("sloy.csv", index_col="year")
table = pd.read_csv("table.csv", index_col='Орган')
tochki_pitanie = pd.read_csv("tochki_pitanie.csv", index_col='Орган').drop('Unnamed: 0', axis=1)
channels_Ke = pd.read_csv("channels_Ke.csv", index_col=0)
points = pd.read_csv("points.csv", index_col="Точка")#.drop('Unnamed: 0', axis=1)
pitanie = pd.read_csv("pitanie.csv", index_col='Unnamed: 0')
ke = pd.read_csv("ke.csv", index_col='Unnamed: 0')

stihiya = {"вода":"металл", "дерево":"вода", "огонь":"дерево", "земля":"огонь", "металл":"земля"}
stvoly_Ke = {"Lu":"Si", "Co":"Liv", "Sp":"Gb", "St":"Kid", "Ht":"Bl"}
vetvi_Ke = {"Lu":"Bl", "Co":"Kid", "Sp":"Th", "St":"Hg", "Ht":"Gb", "Si":"Liv"}
u_sin_pitanie = {'Ht': 'Liv',
                'Si': 'Gb',
                'Hg': 'Liv',
                'Th': 'Gb',
                'Sp': 'Hg',
                'St': 'Th',
                'Lu': 'Sp',
                'Co': 'St',
                'Kid': 'Lu',
                'Bl': 'Co',
                'Liv': 'Kid',
                'Gb': 'Bl'}
u_sin_Ke = {'Hg': 'Bl',
            'Ht': 'Bl',
            'St': 'Liv',
            'Lu': ['Th','Si'],
            'Co': ['Hg', 'Ht'],
            'Kid': 'St',
            'Bl': 'Sp',
            'Liv': 'Co',
            'Gb': 'Lu',
            'Th': 'Kid',
            'Si': 'Kid',
            'Sp':'Gb'}
home_Ke = {'Liv': 'Gb',
            'Gb': 'Liv',
            'Ht': ['Si', 'Th'],
            'Si': ['Ht', 'Hg'],
            'Hg': ['Si', 'Th'],
            'Th': ['Ht', 'Hg'],
            'Sp': 'St',
            'St': 'Sp',
            'Lu': 'Co',
            'Co': 'Lu',
            'Kid': 'Bl',
            'Bl': 'Kid'}



vis_yaer = [1920, 1924, 1928, 1932, 1936, 1940, 1944, 1948, 1952, 1956, 1960, 1964, 1968, 
            1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 
            2024, 2028, 2032, 2036, 2040, 2044, 2048, 2052]

moon_palace = dict({1: [1920, 1942, 0, 1987, 2009, 2032], 2: [0, 1943, 1965, 1988, 2010, 0], 
    3: [1921, 1944, 1966, 0, 2011, 2033], 4: [1922, 0, 1967, 1989, 2012, 2034], 
    5: [1923, 1945, 1968, 1990, 0, 2035], 6: [1924, 1946, 0, 1991, 2013, 2036], 
    7: [0, 1947, 1969, 1992, 2014, 0], 8: [1925, 1948, 1970, 0, 2015, 2037], 
    9: [1926, 0, 1971, 1993, 2016, 2038], 10: [1927, 1949, 1972, 1994, 0, 2039], 
    11: [1928, 1950, 0, 1995, 2017, 2040], 12: [0, 1951, 1973, 1996, 2018, 0], 
    13: [1929, 1952, 1974, 0, 2019, 2041], 14: [1930, 0, 1975, 1997, 2020, 2042], 
    15: [1931, 1953, 1976, 1998, 0, 2043], 16: [1932, 1954, 0, 1999, 2021, 2044], 
    17: [0, 1955, 1977, 2000, 2022, 0], 18: [1933, 1956, 1978, 0, 2023, 2045], 
    19: [1934, 0, 1979, 2001, 2024, 2046], 20: [1935, 1957, 1980, 2002, 0, 2047], 
    21: [1936, 1958, 0, 2003, 2025, 2048], 22: [0, 1959, 1981, 2004, 2026, 0], 
    23: [1937, 1960, 1982, 0, 2027, 2049], 24: [1938, 0, 1983, 2005, 2028, 2050], 
    25: [1939, 1961, 1984, 2006, 0, 2051], 26: [1940, 1962, 0, 2007, 2029, 2052], 
    27: [0, 1963, 1985, 2008, 2030, 0], 28: [1941, 1964, 1986, 0, 2031, 2053]})

sec_step = {1: 27,
            2: 2,
            3: 2,
            4: 5,
            5: 7,
            6: 10,
            7: 12,
            8: 15,
            9: 18,
            10: 20,
            11: 23,
            12: 25}


def get_Ke(d, value):
    for k, v in d.items():
        if v == value:
            return k
        elif k == value:
            return v


In [209]:
{"t":1, "f":2, "e":3, "m":4, "w":5}


def kachestvo(chan_1, chan_2):
    '''
    Принимает на вход два канала и возвращает их общие качества
    '''
    dict_of_colors = {'Liv':[4,3,1,1], 'Gb':[3,5,1,2], 'Ht':[1,2,2,2], 'Si':[5,3,2,5], 'Hg':[3,2,1], 
                    'Th':[5,2,2], 'Sp':[3,2,3,3], 'St':[2,3,3,4], 'Lu':[5,1,4,3], 'Co':[4,1,4,4], 'Kid':[2,4,5,2], 'Bl':[1,4,5,5]}
    return str(set(dict_of_colors[chan_1]) & set(dict_of_colors[chan_2])).strip('{}')

In [208]:
a = ke["Si"].to_list()
a = " ".join(a[:-1])
a = a.replace(",", '').split()
set(a)

{'Bl', 'Co', 'Hg', 'Ht', 'Kid', 'Liv', 'Lu'}

In [218]:
ke

,Liv,Gb,Ht,Si,Hg,Th,Sp,St,Lu,Co,Kid,Bl
по стволам,Co,Sp,Bl,Lu,-,-,Gb,Kid,Si,Liv,St,Ht
по ветвям,Si,Ht,Gb,Liv,St,Sp,Th,Hg,Bl,Kid,Co,Lu
что по у-син,Co,Lu,Bl,Kid,Bl,Kid,Gb,Liv,"Th, Si","Hg, Ht",St,Sp
внутри дома,Gb,Liv,"Si, Th","Ht, Hg","Si, Th","Ht, Hg",St,Sp,Co,Lu,Bl,Kid
пульс,Sp,St,Lu,Co,Kid,Bl,Liv,Gb,Ht,Si,Hg,Th
слой,Hg,Th,Kid,Bl,Liv,Gb,Lu,Co,Sp,St,Ht,Si
на что по у-син,St,Sp,Co,Lu,Co,Lu,Bl,Kid,Gb,Liv,"Si, Th","Ht, Hg"
по качеству,"Gb, St, Lu, Co, Bl","Liv, Ht, Si, Hg, Th, Sp, St, Lu, Kid, Bl","Gb, Hg, Th, Kid","Gb, Hg, Th, Sp, St, Lu, Kid, Bl","Liv, Gb, Ht, Si, Sp, St, Lu","Gb, Ht, Si, Kid","Gb, Si, Hg, St","Liv, Gb, Si, Hg, Sp, Lu","Liv, Gb, Si, Hg, St, Co, Kid, Bl","Liv, Lu, Bl","Gb, Ht, Si, Th, St, Lu, Bl","Liv, Si, Co, Kid, Bl"


In [214]:
a = ke.loc["по качеству", "Si"].replace(",", '').split()
b = ke.loc["по качеству", "Liv"].replace(",", '').split()

set(a) & set(b)

{'Bl', 'Gb', 'Lu', 'St'}

In [186]:
# ke.to_csv("ke.csv")

In [3]:
points.loc["Kid10", "Локализация"]

'При согнутой в коленном суставе ноге на внутреннем конце подколенной складки, между сухожилиями полуперепончатой и полусухожильной мышц.'

In [91]:
make_gui = "\tИспользуется техника выведения:\
    \nПарные иглы ставятся билатерально в дистперсии под углом 90*\
    \nПоследовательность постановки - по полу: с сильной стороны на слабую. \
    \nОдиночные иглы ставятся в дисперсии под углом 90*"
make_xue = "\tИглы ставятся билатерально в технике `тяни-толкай`:\
    \nПод углом 90* на здоровой стороне в тонизации, на больной - в дисперсии.\
    \nСнимаются в обратном порядке. \
    \nПри двусторонней или срединной травме иглы ставятся по полу:\
    \nна сильной стороне в тонизации, на слабой - в дисперсии."
make_pit = "Иглы ставятся в тонизации, по полу, на слабой стороне:\
    \nдля мужчин - слева, для женщин - справа"

In [92]:
print(make_pit)

Иглы ставятся в тонизации по полу на слабой стороне:    
для мужчин - слева, для женщин - справа


In [142]:
di = {'+':['Bl', 'St'],
    '__':['Kid', 'Liv'],
    'Gui':['Ht', 'Gb'],
    'Xue':['Kid', 'Co'],
    'жар':['Ht, Liv'],
    'холод':[],
    'сырость':[],
    'сухость':[]
    }

df_tochki = pd.DataFrame(
            data='',
            index=list(di.keys())[2:],
            columns=["Точки", "Техника постановки"]
        )

for i in di['Gui']:
    df_tochki.loc['Gui', 'Точки'] += table.loc[i, 'Gui'] + ', '
gui_toch = df_tochki.loc['Gui', 'Точки']
gui_toch = gui_toch.strip(', ').split(', ')
gui_toch = ', '.join(list(set(gui_toch)))
df_tochki.loc['Gui', 'Точки'] = gui_toch
df_tochki.loc['Gui', 'Техника постановки'] = make_gui

for i in di['Xue']:
    df_tochki.loc['Xue', 'Точки'] += table.loc[i, 'Xue'] + ', '
gui_toch = df_tochki.loc['Xue', 'Точки']
gui_toch = gui_toch.strip(', ').split(', ')
gui_toch = ', '.join(list(set(gui_toch)))
df_tochki.loc['Xue', 'Точки'] = gui_toch
df_tochki.loc['Xue', 'Техника постановки'] = make_xue

for k in list(di.keys())[2:]:
    if di[k]:
        print(". "*40)
        print(f"{k.capitalize()}: {df_tochki.loc[k, 'Точки']}")
        print(". "*40)
        print(f"{df_tochki.loc[k, 'Техника постановки']}")
        print("__"*40)

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Gui: Lu11, Gv23
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
**Выведение:**
Парные иглы ставятся билатерально в дистперсии под углом 90* 
Последовательность постановки - по полу: с сильной стороны на слабую. 
Одиночные иглы ставятся в дисперсии под углом 90*
________________________________________________________________________________
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Xue: Co7, Kid5
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
**P & P:**
 
________________________________________________________________________________
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Жар: 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

________________________________________________________________________________


# **Лунные дворцы**

In [138]:
# Вводим дату:

try:
    date = str(pd.to_datetime(input("Введите дату рождения"), dayfirst=True)).split()[0]
    print(date)
    year = int(date[:4])
    month = int(date[5:7])
    day = int(date[8:])
except:
    print("Некорректная дата. Попробуйте снова")


Некорректная дата. Попробуйте снова


In [10]:
for k, v in moon_palace.items():
    if year in v:
        first_step = k

if (year in vis_yaer) & (pd.to_datetime(date) > pd.to_datetime(f"{year}-02-28")):
    first_step = first_step+1

lunar_day = first_step + sec_step[month]+ day

while lunar_day > 28:
    lunar_day+=-28
print(f"Лунный день по дате: {lunar_day}")

NameError: name 'year' is not defined

In [ ]:
table.loc["Sp", ["Jing_Jin"]] = "Sp1"
table

,Стихия_усин,Инь_Ян,Стихия_сезон,Gui,Xi,Luo,Jing_Jin,Yuan
Орган,,,,,,,,
Gb,дерево,Yang,вода,Gv23,Gb36,Gb37,Gb44,Gb40
Si,огонь,Yang,земля,HG8(HG7),Si6,Si7,Si1,Si4
Th,огонь,Yang,вода,Sp1,Th7,Th5,Th1,Th4
St,земля,Yang,земля,St6,St34,St40,St45,St42
Co,металл,Yang,дерево,Cv24,Co7,Co6,Co1,Co4
Bl,вода,Yang,металл,HG7(HG5),Bl63,Bl58,Bl67,Bl64
Liv,дерево,Yin,земля,Gv16,Liv6,Liv5,Liv1,Liv3
Ht,огонь,Yin,огонь,Lu11,Ht6,Ht5,Ht9,Ht7
Hg,огонь,Yin,земля,Bl62,Hg4,Hg6,Hg9,Hg7


In [ ]:
table = table.rename(columns={"Xi":"Xue"})
table

,Стихия_усин,Инь_Ян,Стихия_сезон,Gui,Xue,Luo,Jing_Jin,Yuan
Орган,,,,,,,,
Gb,дерево,Yang,вода,Gv23,Gb36,Gb37,Gb44,Gb40
Si,огонь,Yang,земля,HG8(HG7),Si6,Si7,Si1,Si4
Th,огонь,Yang,вода,Sp1,Th7,Th5,Th1,Th4
St,земля,Yang,земля,St6,St34,St40,St45,St42
Co,металл,Yang,дерево,Cv24,Co7,Co6,Co1,Co4
Bl,вода,Yang,металл,HG7(HG5),Bl63,Bl58,Bl67,Bl64
Liv,дерево,Yin,земля,Gv16,Liv6,Liv5,Liv1,Liv3
Ht,огонь,Yin,огонь,Lu11,Ht6,Ht5,Ht9,Ht7
Hg,огонь,Yin,земля,Bl62,Hg4,Hg6,Hg9,Hg7


In [ ]:
# table.to_csv("table.csv")

In [ ]:
sex = "Мужчина"

In [ ]:

man = ["Liv1", "Liv4", "Liv3", "Liv3, Gb37", "Liv5, Gb40", "Liv2", "Liv8", 
       "Kid1", "Kid7", "Kid3", "Kid3, Bl58", "Kid4, Bl64", "Kid2", "Kid10", 
       "Lu11", "Lu8", "Lu9", "Lu9, Co6", "Lu7, Co4", "Lu10", "Lu5", 
       "Hg9, Ht9", "Hg5, Ht4", "Hg7, Ht7", "Hg7, Ht7, Si7", "Hg6, Ht5, Si4", "Hg8, Ht8", "Hg3, Ht3"]

woman = ["Gb41", "Gb44", "Gb34", "Gb37, Liv3", "Gb40, Liv5", "Gb38", "Gb43", 
       "Bl65", "Bl67", "Bl40", "Kid3, Bl58", "Kid4, Bl64", "Bl60", "Bl66", 
       "Co3", "Co1", "Co11", "Lu9, Co6", "Lu7, Co4", "Co5", "Co2", 
       "Si3", "Si1", "Si8", "Hg7, Ht7, Si7", "Ht5, Si4", "Si5", "Si2"]

